# 📦 Imports

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import ta

# ✅ Check Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


# 🧪 Load Data

In [3]:
def load_data(csv_file, ticker_id=None):
    df = pd.read_csv(csv_file, delimiter='\t')
    df.columns = df.columns.str.strip()
    df['Gmt time'] = pd.to_datetime(df['Gmt time'], format='%d.%m.%Y %H:%M:%S.%f')
    df.sort_values('Gmt time', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['Ticker'] = os.path.basename(csv_file).split('_')[0]
    if ticker_id is not None:
        df['Ticker_ID'] = ticker_id
    return df

# ➕ Add Technical Indicators

In [4]:
def add_technical_indicators(df):
    df['rsi'] = ta.momentum.rsi(df['Close'], window=14)
    bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
    df['bb_high'] = bb.bollinger_hband()
    df['bb_low'] = bb.bollinger_lband()
    df['ma_20'] = df['Close'].rolling(20).mean()
    df['ma_20_slope'] = df['ma_20'].diff()
    df['macd'] = ta.trend.macd_diff(df['Close'])
    df['ema_12'] = ta.trend.ema_indicator(df['Close'], window=12)
    df['ema_26'] = ta.trend.ema_indicator(df['Close'], window=26)
    df['atr'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'])
    df['adx'] = ta.trend.adx(df['High'], df['Low'], df['Close'])
    df['cci'] = ta.trend.cci(df['High'], df['Low'], df['Close'])
    df['obv'] = ta.volume.on_balance_volume(df['Close'], df['Volume'])
    df['target_up'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.fillna(method='bfill', inplace=True)
    df.fillna(method='ffill', inplace=True)
    return df


# 🔄 Feature Selection and Scaling

In [5]:
def select_and_scale_features(df):
    feature_cols = [
        'Open', 'High', 'Low', 'Close', 'Volume', 'rsi', 'bb_high', 'bb_low', 'ma_20',
        'ma_20_slope', 'macd', 'ema_12', 'ema_26', 'atr', 'adx', 'cci', 'obv', 'Ticker_ID'
    ]
    data = df[feature_cols].values
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)
    return data_scaled, scaler, feature_cols


# 📚 Dataset Class

In [6]:

class StockDataset(Dataset):
    def __init__(self, data, targets, seq_length=60, pred_length=1):
        self.data = data
        self.targets = targets
        self.seq_length = seq_length
        self.pred_length = pred_length

    def __len__(self):
        return len(self.data) - self.seq_length - self.pred_length + 1

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.seq_length]
        y_price = self.data[idx+self.seq_length:idx+self.seq_length+self.pred_length, 3]  # Close price
        y_dir = self.targets[idx+self.seq_length+self.pred_length-1]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y_price, dtype=torch.float32), torch.tensor(y_dir, dtype=torch.float32)


# 🧠 Time Series Transformer

In [7]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, seq_length, pred_length):
        super().__init__()
        self.d_model = 64
        self.input_fc = nn.Linear(input_dim, self.d_model)
        self.pos_embedding = nn.Parameter(torch.zeros(1, seq_length, self.d_model))
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.d_model, nhead=8, dim_feedforward=256,batch_first=True )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.price_head = nn.Linear(self.d_model, pred_length)
        self.class_head = nn.Linear(self.d_model, 1)

    def forward(self, x):
        x = self.input_fc(x) + self.pos_embedding
        x = x.permute(1, 0, 2)
        x = self.encoder(x)
        last_step = x[-1]
        return self.price_head(last_step), self.class_head(last_step).squeeze(-1)


# 🏋️ Training Loop

In [8]:
def train(model, train_loader, val_loader, epochs=200, lr=1e-3):
    criterion_price = nn.MSELoss()
    criterion_class = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(epochs):
        model.train()
        losses = []
        for x, y_price, y_dir in train_loader:
            x, y_price, y_dir = x.to(device), y_price.to(device), y_dir.to(device)
            pred_price, pred_dir = model(x)
            loss = criterion_price(pred_price, y_price) + criterion_class(pred_dir, y_dir)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        print(f"Epoch {epoch+1}: Train Loss = {np.mean(losses):.4f}")
    
    return model

# 📈 Evaluation

In [9]:
@torch.no_grad()
def evaluate(model, loader, scaler, feature_cols):
    model.eval()
    target_idx = feature_cols.index('Close')
    preds, actuals = [], []
    for x, y_price, _ in loader:
        x = x.to(device)
        price_pred, _ = model(x)
        for i in range(len(price_pred)):
            dummy = np.zeros((price_pred.shape[1], len(feature_cols)))
            dummy[:, target_idx] = price_pred[i].cpu().numpy()
            inv = scaler.inverse_transform(dummy)
            preds.extend(inv[:, target_idx])
            actuals.extend(scaler.inverse_transform(np.insert(np.zeros((y_price.shape[1], len(feature_cols)-1)), target_idx, y_price[i].cpu().numpy(), axis=1))[:, target_idx])
    plt.figure(figsize=(12,6))
    plt.plot(preds, label='Predicted')
    plt.plot(actuals, label='Actual', alpha=0.7)
    plt.legend(); plt.grid(True); plt.title("Real vs Predicted Close Price"); plt.show()


# 🧪 Main Entry

In [10]:
DATA_DIR = '../Data/formatted_data_1h'
csv_files = [os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
ticker2id = {os.path.basename(f).split('_')[0]: i for i, f in enumerate(csv_files)}


In [11]:
all_df = pd.concat([add_technical_indicators(load_data(f, ticker_id=ticker2id[os.path.basename(f).split('_')[0]])) for f in csv_files])
data_scaled, scaler, feature_cols = select_and_scale_features(all_df)
targets = all_df['target_up'].values


/tmp/ipykernel_5882/77177142.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
/tmp/ipykernel_5882/77177142.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/tmp/ipykernel_5882/77177142.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
/tmp/ipykernel_5882/77177142.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/tmp/ipykernel_5882/77177142.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffil

In [12]:
seq_length, pred_length, batch_size = 60, 1, 32
dataset = StockDataset(data_scaled, targets, seq_length, pred_length)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
train_loader = DataLoader(Subset(dataset, range(train_size)), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(Subset(dataset, range(train_size, train_size + val_size)), batch_size=batch_size)
test_loader = DataLoader(Subset(dataset, range(train_size + val_size, len(dataset))), batch_size=batch_size)


In [13]:

model = TimeSeriesTransformer(input_dim=len(feature_cols), seq_length=seq_length, pred_length=pred_length)
model = train(model, train_loader, val_loader)
evaluate(model, test_loader, scaler, feature_cols)

Epoch 1: Train Loss = 0.6933
Epoch 2: Train Loss = 0.6931
Epoch 3: Train Loss = 0.6931
Epoch 4: Train Loss = 0.6931


KeyboardInterrupt: 

now wait 

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import torch

def evaluate_and_plot(model, test_loader, scaler, feature_cols, target_col_idx,
                      window_width=50, start_index=0, pred_length=1, device='cpu'):
    model.eval()
    real_prices = []
    predicted_prices = []

    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            predictions = model(x_batch)
            predictions = predictions.cpu().numpy()
            y_batch = y_batch.cpu().numpy()

            for i in range(len(predictions)):
                dummy_pred = np.zeros((pred_length, len(feature_cols)))
                dummy_pred[:, target_col_idx] = predictions[i]

                dummy_real = np.zeros((pred_length, len(feature_cols)))
                dummy_real[:, target_col_idx] = y_batch[i]

                pred_inversed = scaler.inverse_transform(dummy_pred)[:, target_col_idx]
                real_inversed = scaler.inverse_transform(dummy_real)[:, target_col_idx]

                predicted_prices.extend(pred_inversed)
                real_prices.extend(real_inversed)

    real_prices = np.array(real_prices).flatten()
    predicted_prices = np.array(predicted_prices).flatten()

    mse = np.mean((real_prices - predicted_prices) ** 2)
    mae = np.mean(np.abs(real_prices - predicted_prices))

    print(f"📊 Model Evaluation:")
    print(f"  - Mean Squared Error (MSE): {mse:.4f}")
    print(f"  - Mean Absolute Error (MAE): {mae:.4f}")

    # 🖼️ Plot
    plt.figure(figsize=(12, 6))
    end_index = min(start_index + window_width, len(real_prices))
    plt.plot(range(start_index, end_index), real_prices[start_index:end_index], label="Real", linestyle='--', marker='o')
    plt.plot(range(start_index, end_index), predicted_prices[start_index:end_index], label="Predicted", linestyle='-', marker='x')
    plt.title(f"Predicted vs Real Close Prices (index {start_index} to {end_index})")
    plt.xlabel("Time Step")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [20]:
# ✅ Define target column index for 'Close'
target_col_idx = feature_cols.index('Close')

evaluate_and_plot(
    model,
    test_loader,
    scaler,
    feature_cols,
    target_col_idx,
    window_width=60,
    start_index=100,
    pred_length=1,
    device=device
)


ValueError: too many values to unpack (expected 2)

In [ ]:
# 📦 Save trained model
model_save_path = "../Models/trained_trade_transformer1_more_dataset.pth"
torch.save(model.state_dict(), model_save_path)
print(f"✅ Model saved to: {model_save_path}")


✅ Model saved to: ../Models/trained_trade_transformer1_more_epoch.pth
